# Russian Synodal Bible (1885)

## Import

### Libraries

In [1]:
import os
import re
import json
import time
import numpy as np
import pandas as pd 
from lxml import etree
import natasha
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc
#from bs4 import BeautifulSoup
#import requests

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [53]:
texts = './texts/fiction/'
bibleTXT = './texts/bible/sinodalnyi-perevod.txt'
libCols = ['author','pub_year','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']
bibleOHCO = ['test', 'book', 'chap', 'verse']
bibleJSON = './texts/bible/bible.json'
bibleXML = './texts/bible/bible.xml'
tokenCols = ['v_id', 'token_num', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

## Pre-Processing

### Import Text into DF

In [122]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

,text
0,
1,== Бытие ==
2,
3,=== 1 ===
4,
...,...
41458,17 И Дух и невеста говорят: прииди! И слышавши...
41459,18 И я также свидетельствую всякому слышащему ...
41460,19 и если кто отнимет что от слов книги пророч...
41461,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Find Parts

In [123]:
chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains('^===\s\d{1,3}\s===$')]
book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains('^==\s.+\s==$')]

### Tidy Up

In [137]:
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '']
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')
bibliiaDf

/tmp/ipykernel_253881/185640697.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
/tmp/ipykernel_253881/185640697.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')


,text
1,== Бытие ==
3,=== 1 ===
5,1 В начале сотворил Бог небо и землю.
6,"2 Земля же была безвидна и пуста, и тьма над б..."
7,3 И сказал Бог: да будет свет. И стал свет.
...,...
41458,17 И Дух и невеста говорят: прииди! И слышавши...
41459,18 И я также свидетельствую всякому слышащему ...
41460,19 и если кто отнимет что от слов книги пророч...
41461,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Make Dict of Outline

{Testament: {Book: abbr}}

In [139]:
book_lines

,text
1,== Бытие ==
1685,== Исход ==
3021,== Левит ==
3962,== Числа ==
5360,== Второзаконие ==
...,...
40461,== Второе послание к Тимофею святого апостола ...
40559,== Послание к Титу святого апостола Павла ==
40617,== Послание к Филимону святого апостола Павла ==
40648,== Послание к Евреям святого апостола Павла ==


In [ ]:
abbrevsDf = bibliiaDf.loc[vz_line+1:OT_line-1]
abbrevsDf[['abbr', 'title']] = abbrevsDf.loc[:,'text'].str.extract(r'^(\[.*\]) (.*)$', expand=True)
abbrevsDf = abbrevsDf.drop(['text'], axis=1)
bibliiaDict = {
    'Ветхий Завет': dict(zip(abbrevsDf.loc[vz_line+1:nz_line-1].title.str.strip(' '), abbrevsDf.loc[vz_line+1:nz_line-1].abbr.str.strip(r'[|]'))), 
    'Новый Завет': dict(zip(abbrevsDf.loc[nz_line+1:OT_line-1].title.str.strip(' '), abbrevsDf.loc[nz_line+1:OT_line-1].abbr.str.strip(r'[|]')))
}
del abbrevsDf

### Assign Testament Label

In [ ]:
bibliiaDf = bibliiaDf.loc[OT_line+1:end_line-1]
bibliiaDf['test'] = np.where(bibliiaDf.index<NT_line, 'OT', 'NT')
bibliiaDf

### Index All Parts

In [ ]:
all_chaps = bibliiaDf.loc[bibliiaDf.text.str.contains(r'Глава \d')]
bibleBooks = [x for y in bibliiaDict.values() for x in y]
bookAbbrs = [val[abbr] for key, val in bibliiaDict.items() for abbr in val]
searchAbbrs = [r'^\['+val[abbr]+'\]' for key, val in bibliiaDict.items() for abbr in val]
abbrsList = "|".join(searchAbbrs)
books_index = bibliiaDf[bibliiaDf['text'].str.contains(abbrsList)].index
books_map = list(zip(books_index, bookAbbrs))
chaps_map = list(all_chaps.itertuples(name=None))

### Assign Book Label

In [ ]:
for ind in books_map: 
    bibliiaDf.loc[ind[0], 'book'] = ind[1]

bibliiaDf['book'] = bibliiaDf.book.ffill()
bibliiaDf = bibliiaDf.drop([x[0] for x in books_map])
bibliiaDf

### Assign Chapter & Verse Labels

### Set Index & Remove Fluff

### Export to JSON

### Make XML

In [ ]:
root = etree.Element("bible")
print(root)
print(root.tag)
#bibleTree = etree.ElementTree()

In [ ]:
for v in BibDf.index.to_list(): 
    

In [ ]:
BibDf

In [ ]:
root.append(etree.Element("testament"))

In [ ]:
root.append(etree.Element(""))

In [ ]:
etree.tostring(root, pretty_print=True)

## Processing

In [4]:
BibDf = pd.read_json(bibleJSON)
BibDf.index.name = 'v_id'
BibTextDf = BibDf[['text']]
BibLibDf = BibDf[bibleOHCO]
#synodalDf#.set_index(bibleOHCO)

In [5]:
BibDf

,test,book,chap,verse,text
v_id,,,,,
0,OT,Быт.,1,1,В начале сотворил Бог небо и землю.
1,OT,Быт.,1,2,"Земля же была безвидна и пуста, и тьма над без..."
2,OT,Быт.,1,3,И сказал Бог: да будет свет. И стал свет.
3,OT,Быт.,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
4,OT,Быт.,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37338,NT,Откр.,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37339,NT,Откр.,22,18,И я также свидетельствую всякому слышащему сло...
37340,NT,Откр.,22,19,и если кто отнимет что от слов книги пророчест...


In [6]:
#testsDict = dict(enumerate(BibDf.test.unique()))
testsDict = dict([(value, key) for key, value in dict(enumerate(BibDf.test.unique())).items()])
booksDict = dict([(value, key) for key, value in dict(enumerate(BibDf.book.unique())).items()])

In [7]:
TestsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:1]).text], columns=['test', 'text'])
TestsDf.index.name = 't_id'
#TestsDf 

In [8]:
BooksDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:2]).text], columns=[('test', 'book'), 'text'])
BooksDf[['test','book']] = pd.DataFrame(list(BooksDf[('test', 'book')]), index=BooksDf.index, columns=bibleOHCO[:2])
del BooksDf[('test', 'book')]
BooksDf = BooksDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:2], ascending=[True, True])
BooksDf = BooksDf.reset_index().drop(['index'], axis=1)
BooksDf.index.name = 'b_id'
BooksDf = BooksDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#BooksDf = BooksDf.reset_index().set_index(bibleOHCO[:2])
#BooksDf 

In [74]:
ChapsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:3]).text], columns=[('test', 'book', 'chap'), 'text'])
ChapsDf[['test','book','chap']] = pd.DataFrame(list(ChapsDf[('test', 'book', 'chap')]), index=ChapsDf.index, columns=bibleOHCO[:3])
del ChapsDf[('test', 'book', 'chap')]
ChapsDf = ChapsDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:3], ascending=[True, True, True])
ChapsDf = ChapsDf.reset_index().drop(['index'], axis=1)
ChapsDf.index.name = 'c_id'
ChapsDf = ChapsDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#ChapsDf = ChapsDf.reset_index().set_index(bibleOHCO[:3])
ChapsDf

,text,test,book,chap
c_id,,,,
0,В начале сотворил Бог небо и землю. Земля же б...,OT,Быт.,1
1,Так совершены небо и земля и все воинство их. ...,OT,Быт.,2
2,"Змей был хитрее всех зверей полевых, которых с...",OT,Быт.,3
3,"Адам познал Еву, жену свою; и она зачала, и ро...",OT,Быт.,4
4,Вот родословие Адама: когда Бог сотворил челов...,OT,Быт.,5
...,...,...,...,...
1193,"После сего я увидел иного Ангела, сходящего с ...",NT,Откр.,18
1194,После сего я услышал на небе громкий голос как...,NT,Откр.,19
1195,"И увидел я Ангела, сходящего с неба, который и...",NT,Откр.,20


In [13]:
BibTextDf.text

v_id
0                      В начале сотворил Бог небо и землю.
1        Земля же была безвидна и пуста, и тьма над без...
2                И сказал Бог: да будет свет. И стал свет.
3        И увидел Бог свет, что он хорош, и отделил Бог...
4        И назвал Бог свет днем, а тьму ночью. И был ве...
                               ...                        
37338    И Дух и невеста говорят: прииди! И слышавший д...
37339    И я также свидетельствую всякому слышащему сло...
37340    и если кто отнимет что от слов книги пророчест...
37341    Свидетельствующий сие говорит: ей, гряду скоро...
37342    Благодать Господа нашего Иисуса Христа со всем...
Name: text, Length: 37343, dtype: object

In [45]:
def nat_parse(textDf=BibTextDf, textCol='text', columns=tokenCols): 
    tokenDf = pd.DataFrame(columns=columns)
    for v_id in textDf.index: 
        verseDict = []
        doc = Doc(textDf.loc[v_id][textCol])
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens: 
            token.lemmatize(morph_vocab)
        doc.parse_syntax(syntax_parser)
        doc.tag_ner(ner_tagger)
        for sent in enumerate(doc.sents): 
            sent_num = sent[0]
            sent_text = sent[1]
            for token in enumerate([x for x in sent_text.tokens if x.pos!='PUNCT']): 
                token_num = token[0]
                token_text = token[1]
                start = token_text.start
                stop = token_text.stop
                text = token_text.text
                token_id = token_text.id
                head_id = token_text.head_id
                rel = token_text.rel
                pos = token_text.pos
                lemma = token_text.lemma
                # Animacy, Aspect, Case, Degree, Gender, Mood, Number, Person, Tense, VerbForm, Voice
                #print(token_text.feats)
                try: 
                    anim = token_text.feats['Animacy']
                except: 
                    anim = None
                try: 
                    aspect = token_text.feats['Aspect']
                except: 
                    aspect = None
                try: 
                    case = token_text.feats['Case']
                except: 
                    case = None
                try: 
                    degree = token_text.feats['Degree']
                except: 
                    degree = None
                try: 
                    gender = token_text.feats['Gender']
                except: 
                    gender = None
                try: 
                    mood = token_text.feats['Mood']
                except: 
                    mood = None
                try: 
                    number = token_text.feats['Number']
                except: 
                    number = None
                try: 
                    person = token_text.feats['Person']
                except: 
                    person = None
                try: 
                    tense = token_text.feats['Tense']
                except: 
                    tense = None
                try: 
                    verb_form = token_text.feats['VerbForm']
                except: 
                    verb_form = None
                try: 
                    voice = token_text.feats['Voice']
                except: 
                    voice = None
                #print(token)
                tokenDict = {
                    'v_id': v_id,
                    'token_num': token_num, 
                    'start': start, 
                    'stop': stop, 
                    'text': text, 
                    'token_id': token_id, 
                    'head_id': head_id, 
                    'rel': rel, 
                    'pos': pos, 
                    'lemma': lemma, 
                    'anim': anim, 
                    'aspect': aspect, 
                    'case': case, 
                    'degree': degree, 
                    'gender': gender, 
                    'mood': mood, 
                    'number': number, 
                    'person': person, 
                    'tense': tense, 
                    'verb_form': verb_form, 
                    'voice': voice
                }
                verseDict.append(tokenDict)
            #print(sent)
            verseDf = pd.DataFrame(verseDict, columns=columns)
        tokenDf = pd.concat([tokenDf, verseDf])
    return tokenDf

In [51]:
%%time
tokenDf = nat_parse(BibTextDf).set_index(['v_id', 'token_num'])

KeyboardInterrupt: 

In [52]:
tokenDf.to_pickle('./proc/tokenDf.pkl')

NameError: name 'tokenDf' is not defined